In [ ]:
import numpy as np
import cvxpy as cp

In [ ]:
def generate_test(N=1000, S=10, M=100, K=10):
    """
    # Construct the problem
    N = 1000 # Population size
    S = 10 # Number of infected individuals
    M = 100 # Number of tests
    K = 10 # Number of splits of each sample
    """

    # Define x0
    ind0 = np.random.choice(N,S,0) # index subset 
    x0 = np.zeros(N) 
    x0[ind0] = np.random.rand(S)

    # Define A
    A = np.zeros((M,N))
    for i in np.arange(N):
        ind = np.random.choice(M,K,replace=False)
        A[ind,i] = 1

    y = A @ x0

    return y, A, x0

In [2]:
N=1000
S=10
M=100
K=10

## a)

In [3]:
def estimate_test(N=1000, S=10, M=100, K=10):
    y, A, x0 = generate_test(N=N, S=S, M=M, K=K)

    x = cp.Variable(N)

    objective = cp.Minimize(cp.norm1(x))

    constraints = [
        x >= 0,
        A @ x == y
    ]

    problem = cp.Problem(objective, constraints)

    problem.solve()

    return x.value, x0

In [4]:
x, x0 = estimate_test()

if np.linalg.norm(x - x0) < 1e-6:
    print(f'solver result and x0 are the same ||x - x0|| = {np.linalg.norm(x - x0)}')
else:
    print(f'solver result and x0 are not the same |x - x0|| = {np.linalg.norm(x - x0)}')

solver result and x0 are the same ||x - x0|| = 1.1272696657227958e-08


## b)

In [5]:
def find_lowestK(treshold=1e-6):
    for k in range(K, 0, -1):
        x, x0 = estimate_test(K=k)
        if np.linalg.norm(x - x0) > treshold:
            return k+1
    return None

In [6]:
treshold=1e-6

lowest_k = find_lowestK()

print(f'lowerst K such that ||x - x0|| < {treshold} is {lowest_k}')

lowerst K such that ||x - x0|| < 1e-06 is 3


## c)

In [7]:
def find_largestS(s=S, k=K, treshold=1e-6, max_iter=1000):
    for _ in range(max_iter):
        x, x0 = estimate_test(S=s, K=k)
        if np.linalg.norm(x - x0) > treshold:
            return s-1
        s += 1
    return None

In [8]:
largest_s = find_largestS()

print(f'largest S such that ||x - x0|| < {treshold} is {largest_s}')

largest S such that ||x - x0|| < 1e-06 is 25


In [9]:
def find_bestK(K_list):
    errors = []
    for k in K_list:
        best_S = find_largestS(k=k)
        x, x0 = estimate_test(S=best_S, K=k)
        errors.append(np.linalg.norm(x - x0))

    best_k_index = np.argmin(errors)

    return K_list[best_k_index], errors[best_k_index]

In [ ]:
K_list = [i for i in range(10, 50)]

best_k, best_k_error = find_bestK(K_list)

print(f'best K is {best_k} with ||x - x0||={np.round(best_k_error, 4)}')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/cvxpy/problems/problem.py:1481: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


best K is 24 with ||x - x0||=0.0
